# Credit Risk Resampling Techniques

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Read the CSV and Perform Basic Data Cleaning

In [4]:
columns = [
    "loan_amnt", "int_rate", "installment", "home_ownership",
    "annual_inc", "verification_status", "issue_d", "loan_status",
    "pymnt_plan", "dti", "delinq_2yrs", "inq_last_6mths",
    "open_acc", "pub_rec", "revol_bal", "total_acc",
    "initial_list_status", "out_prncp", "out_prncp_inv", "total_pymnt",
    "total_pymnt_inv", "total_rec_prncp", "total_rec_int", "total_rec_late_fee",
    "recoveries", "collection_recovery_fee", "last_pymnt_amnt", "next_pymnt_d",
    "collections_12_mths_ex_med", "policy_code", "application_type", "acc_now_delinq",
    "tot_coll_amt", "tot_cur_bal", "open_acc_6m", "open_act_il",
    "open_il_12m", "open_il_24m", "mths_since_rcnt_il", "total_bal_il",
    "il_util", "open_rv_12m", "open_rv_24m", "max_bal_bc",
    "all_util", "total_rev_hi_lim", "inq_fi", "total_cu_tl",
    "inq_last_12m", "acc_open_past_24mths", "avg_cur_bal", "bc_open_to_buy",
    "bc_util", "chargeoff_within_12_mths", "delinq_amnt", "mo_sin_old_il_acct",
    "mo_sin_old_rev_tl_op", "mo_sin_rcnt_rev_tl_op", "mo_sin_rcnt_tl", "mort_acc",
    "mths_since_recent_bc", "mths_since_recent_inq", "num_accts_ever_120_pd", "num_actv_bc_tl",
    "num_actv_rev_tl", "num_bc_sats", "num_bc_tl", "num_il_tl",
    "num_op_rev_tl", "num_rev_accts", "num_rev_tl_bal_gt_0",
    "num_sats", "num_tl_120dpd_2m", "num_tl_30dpd", "num_tl_90g_dpd_24m",
    "num_tl_op_past_12m", "pct_tl_nvr_dlq", "percent_bc_gt_75", "pub_rec_bankruptcies",
    "tax_liens", "tot_hi_cred_lim", "total_bal_ex_mort", "total_bc_limit",
    "total_il_high_credit_limit", "hardship_flag", "debt_settlement_flag"
]

target = ["loan_status"]

In [5]:
# Load the data
file_path = Path('/content/drive/MyDrive/Colab Notebooks/Data Bootcamp/Module 17 /Challenge/LoanStats_2019Q1.csv')
df = pd.read_csv(file_path, skiprows=1)[:-2]
df = df.loc[:, columns].copy()

# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')

# Drop the null rows
df = df.dropna()

# Remove the `Issued` loan status
issued_mask = df['loan_status'] != 'Issued'
df = df.loc[issued_mask]

# convert interest rate to numerical
df['int_rate'] = df['int_rate'].str.replace('%', '')
df['int_rate'] = df['int_rate'].astype('float') / 100


# Convert the target column values to low_risk and high_risk based on their values
x = {'Current': 'low_risk'}   
df = df.replace(x)

x = dict.fromkeys(['Late (31-120 days)', 'Late (16-30 days)', 'Default', 'In Grace Period'], 'high_risk')    
df = df.replace(x)

df.reset_index(inplace=True, drop=True)

df.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,RENT,66000.0,Source Verified,Mar-2019,low_risk,n,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N
1,25000.0,0.2000,929.09,MORTGAGE,105000.0,Verified,Mar-2019,low_risk,n,20.23,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N
2,20000.0,0.2000,529.88,MORTGAGE,56000.0,Verified,Mar-2019,low_risk,n,24.26,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N
3,10000.0,0.1640,353.55,RENT,92000.0,Verified,Mar-2019,low_risk,n,31.44,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,N
4,22000.0,0.1474,520.39,MORTGAGE,52000.0,Not Verified,Mar-2019,low_risk,n,18.76,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,N


In [ ]:
df.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68817 entries, 0 to 68816
Data columns (total 86 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   loan_amnt                   68817 non-null  float64
 1   int_rate                    68817 non-null  float64
 2   installment                 68817 non-null  float64
 3   home_ownership              68817 non-null  object 
 4   annual_inc                  68817 non-null  float64
 5   verification_status         68817 non-null  object 
 6   issue_d                     68817 non-null  object 
 7   loan_status                 68817 non-null  object 
 8   pymnt_plan                  68817 non-null  object 
 9   dti                         68817 non-null  float64
 10  delinq_2yrs                 68817 non-null  float64
 11  inq_last_6mths              68817 non-null  float64
 12  open_acc                    68817 non-null  float64
 13  pub_rec                     688

In [ ]:
df['issue_d'].value_counts()

Jan-2019    31041
Feb-2019    25579
Mar-2019    12197
Name: issue_d, dtype: int64

In [6]:
df_encoded = pd.get_dummies(df, columns=["home_ownership",
                                            "verification_status",
                                            "pymnt_plan",
                                            "application_type",
                                            "hardship_flag",
                                            "debt_settlement_flag",
                                            "initial_list_status",
                                            "next_pymnt_d"])

months = {
    'Jan-2019': 1,
    'Feb-2019': 2,
    'Mar-2019':3
}

df_encoded["issue_d_num"] = df_encoded["issue_d"].apply(lambda x: months[x])

df_encoded = df_encoded.drop(["issue_d"], axis=1)

df_encoded.head()

,loan_amnt,int_rate,installment,annual_inc,loan_status,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,...,pymnt_plan_n,application_type_Individual,application_type_Joint App,hardship_flag_N,debt_settlement_flag_N,initial_list_status_f,initial_list_status_w,next_pymnt_d_Apr-2019,next_pymnt_d_May-2019,issue_d_num
0,10500.0,0.1719,375.35,66000.0,low_risk,27.24,0.0,0.0,8.0,0.0,...,1,1,0,1,1,0,1,0,1,3
1,25000.0,0.2000,929.09,105000.0,low_risk,20.23,0.0,0.0,17.0,1.0,...,1,1,0,1,1,0,1,0,1,3
2,20000.0,0.2000,529.88,56000.0,low_risk,24.26,0.0,0.0,8.0,0.0,...,1,1,0,1,1,0,1,0,1,3
3,10000.0,0.1640,353.55,92000.0,low_risk,31.44,0.0,1.0,10.0,1.0,...,1,1,0,1,1,0,1,0,1,3
4,22000.0,0.1474,520.39,52000.0,low_risk,18.76,0.0,1.0,14.0,0.0,...,1,1,0,1,1,0,1,0,1,3


# Split the Data into Training and Testing

In [7]:
# Create our features
X = df_encoded.drop('loan_status',1)

# Create our target
y = pd.DataFrame(df_encoded['loan_status'])

In [8]:
X.describe()

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,...,pymnt_plan_n,application_type_Individual,application_type_Joint App,hardship_flag_N,debt_settlement_flag_N,initial_list_status_f,initial_list_status_w,next_pymnt_d_Apr-2019,next_pymnt_d_May-2019,issue_d_num
count,68817.000000,68817.000000,68817.000000,6.881700e+04,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,...,68817.0,68817.000000,68817.000000,68817.0,68817.0,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000
mean,16677.594562,0.127718,480.652863,8.821371e+04,21.778153,0.217766,0.497697,12.587340,0.126030,17604.142828,...,1.0,0.860340,0.139660,1.0,1.0,0.123879,0.876121,0.383161,0.616839,1.726172
std,10277.348590,0.048130,288.062432,1.155800e+05,20.199244,0.718367,0.758122,6.022869,0.336797,21835.880400,...,0.0,0.346637,0.346637,0.0,0.0,0.329446,0.329446,0.486161,0.486161,0.743862
min,1000.000000,0.060000,30.890000,4.000000e+01,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,...,1.0,0.000000,0.000000,1.0,1.0,0.000000,0.000000,0.000000,0.000000,1.000000
25%,9000.000000,0.088100,265.730000,5.000000e+04,13.890000,0.000000,0.000000,8.000000,0.000000,6293.000000,...,1.0,1.000000,0.000000,1.0,1.0,0.000000,1.000000,0.000000,0.000000,1.000000
50%,15000.000000,0.118000,404.560000,7.300000e+04,19.760000,0.000000,0.000000,11.000000,0.000000,12068.000000,...,1.0,1.000000,0.000000,1.0,1.0,0.000000,1.000000,0.000000,1.000000,2.000000
75%,24000.000000,0.155700,648.100000,1.040000e+05,26.660000,0.000000,1.000000,16.000000,0.000000,21735.000000,...,1.0,1.000000,0.000000,1.0,1.0,0.000000,1.000000,1.000000,1.000000,2.000000
max,40000.000000,0.308400,1676.230000,8.797500e+06,999.000000,18.000000,5.000000,72.000000,4.000000,587191.000000,...,1.0,1.000000,1.000000,1.0,1.0,1.000000,1.000000,1.000000,1.000000,3.000000


In [9]:
y.head()

,loan_status
0,low_risk
1,low_risk
2,low_risk
3,low_risk
4,low_risk


In [10]:
# Check the balance of our target values
y['loan_status'].value_counts()

low_risk     68470
high_risk      347
Name: loan_status, dtype: int64

In [11]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

# Oversampling

In this section, you will compare two oversampling algorithms to determine which algorithm results in the best performance. You will oversample the data using the naive random oversampling algorithm and the SMOTE algorithm. For each algorithm, be sure to complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

### Naive Random Oversampling

In [73]:
# Resample the training data with the RandomOversampler
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state=1)
X_resampled, y_resampled = ros.fit_resample(X_train, y_train)
Counter(y_resampled)

Counter({'loan_status': 1})

In [74]:
# Train the Logistic Regression model using the resampled data
from sklearn.linear_model import LogisticRegression
regressor = LogisticRegression(solver='lbfgs', random_state=1)
regressor.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [75]:
# Calculated the balanced accuracy score
y_pred = regressor.predict(X_test)
from sklearn.metrics import balanced_accuracy_score, accuracy_score
bacc = (balanced_accuracy_score(y_test, y_pred))
acc = accuracy_score(y_test,y_pred)
print('Balanced Accuracy Score: ' + str(round(bacc, 4)))
print('Accuracy Score: ' + str(round(acc, 4)))

Balanced Accuracy Score: 0.6706
Accuracy Score: 0.6289


In [76]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_test, y_pred, labels=['high_risk', 'low_risk']))

[[   72    29]
 [ 6356 10748]]


In [77]:
cmtx = pd.DataFrame(
    confusion_matrix(y_test, y_pred, labels=['high_risk', 'low_risk']), 
    index=['Actual High Risk', 'Actual Low Risk'], 
    columns=['Predicted High Risk', 'Predicted Low Risk']
    )


                  Predicted High Risk  Predicted Low Risk
Actual High Risk                   72                  29
Actual Low Risk                  6356               10748


In [78]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.01      0.71      0.63      0.02      0.67      0.45       101
   low_risk       1.00      0.63      0.71      0.77      0.67      0.44     17104

avg / total       0.99      0.63      0.71      0.77      0.67      0.44     17205



In [98]:
print('Naive Random Oversampling')
print('')
print('Balanced Accuracy Score: ' + str(round(bacc, 4)))
print('Accuracy Score: ' + str(round(acc, 4)))
print('')
print(cmtx)
print('')
print(classification_report_imbalanced(y_test, y_pred))

Naive Random Oversampling

Balanced Accuracy Score: 0.6706
Accuracy Score: 0.6289

                  Predicted High Risk  Predicted Low Risk
Actual High Risk                   72                  29
Actual Low Risk                  6356               10748

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.01      0.71      0.63      0.02      0.67      0.45       101
   low_risk       1.00      0.63      0.71      0.77      0.67      0.44     17104

avg / total       0.99      0.63      0.71      0.77      0.67      0.44     17205



### SMOTE Oversampling

In [68]:
# Resample the training data with SMOTE
from imblearn.over_sampling import SMOTE
X_resampled2, y_resampled2 = SMOTE(random_state=1).fit_resample(X_train, y_train)
Counter(y_resampled)

Counter({'loan_status': 1})

In [69]:
# Train the Logistic Regression model using the resampled data
regressor2 = LogisticRegression(random_state=1)
regressor2.fit(X_resampled2, y_resampled2)

LogisticRegression(random_state=1)

In [70]:
# Calculated the balanced accuracy score
y_pred2 = regressor2.predict(X_test)

bacc2 = (balanced_accuracy_score(y_test, y_pred2))
acc2 = accuracy_score(y_test,y_pred2)
print('Balanced Accuracy Score: ' + str(round(bacc2, 4)))
print('Accuracy Score: ' + str(round(acc2, 4)))

Balanced Accuracy Score: 0.6546
Accuracy Score: 0.685


In [71]:
# Display the confusion matrix
print(confusion_matrix(y_test, y_pred2, labels=['high_risk', 'low_risk']))

[[   63    38]
 [ 5381 11723]]


In [72]:
cmtx2 = pd.DataFrame(
    confusion_matrix(y_test, y_pred2, labels=['high_risk', 'low_risk']), 
    index=['Actual High Risk', 'Actual Low Risk'], 
    columns=['Predicted High Risk', 'Predicted Low Risk']
    )
print(cmtx2)

                  Predicted High Risk  Predicted Low Risk
Actual High Risk                   63                  38
Actual Low Risk                  5381               11723


In [66]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred2))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.01      0.62      0.69      0.02      0.65      0.42       101
   low_risk       1.00      0.69      0.62      0.81      0.65      0.43     17104

avg / total       0.99      0.69      0.62      0.81      0.65      0.43     17205



In [99]:
print('SMOTE Oversampling')
print('')
print('Balanced Accuracy Score: ' + str(round(bacc2, 4)))
print('Accuracy Score: ' + str(round(acc2, 4)))
print('')
print(cmtx2)
print('')
print(classification_report_imbalanced(y_test, y_pred2))

SMOTE Oversampling

Balanced Accuracy Score: 0.6546
Accuracy Score: 0.685

                  Predicted High Risk  Predicted Low Risk
Actual High Risk                   63                  38
Actual Low Risk                  5381               11723

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.01      0.62      0.69      0.02      0.65      0.42       101
   low_risk       1.00      0.69      0.62      0.81      0.65      0.43     17104

avg / total       0.99      0.69      0.62      0.81      0.65      0.43     17205



# Undersampling

In this section, you will test an undersampling algorithms to determine which algorithm results in the best performance compared to the oversampling algorithms above. You will undersample the data using the Cluster Centroids algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [79]:
# Resample the data using the ClusterCentroids resampler
# Warning: This is a large dataset, and this step may take some time to complete
from imblearn.under_sampling import ClusterCentroids
cc = ClusterCentroids(random_state=1)
X_resampled3, y_resampled3 = cc.fit_resample(X_train, y_train)

In [80]:
# Train the Logistic Regression model using the resampled data
regressor3 = LogisticRegression(random_state=1)
regressor3.fit(X_resampled3, y_resampled3)

LogisticRegression(random_state=1)

In [81]:
# Calculated the balanced accuracy score
y_pred3 = regressor.predict(X_test)
bacc3 = (balanced_accuracy_score(y_test, y_pred3))
acc3 = accuracy_score(y_test,y_pred3)
print('Balanced Accuracy Score: ' + str(round(bacc3, 4)))
print('Accuracy Score: ' + str(round(acc3, 4)))

Balanced Accuracy Score: 0.6546
Accuracy Score: 0.685


In [82]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred3)

array([[   72,    29],
       [ 6356, 10748]])

In [83]:
cmtx3 = pd.DataFrame(
    confusion_matrix(y_test, y_pred3, labels=['high_risk', 'low_risk']), 
    index=['Actual High Risk', 'Actual Low Risk'], 
    columns=['Predicted High Risk', 'Predicted Low Risk']
    )
print(cmtx3)

                  Predicted High Risk  Predicted Low Risk
Actual High Risk                   72                  29
Actual Low Risk                  6356               10748


In [84]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred3))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.01      0.71      0.63      0.02      0.67      0.45       101
   low_risk       1.00      0.63      0.71      0.77      0.67      0.44     17104

avg / total       0.99      0.63      0.71      0.77      0.67      0.44     17205



In [101]:
print('Cluster Centroid')
print('')
print('Balanced Accuracy Score: ' + str(round(bacc3, 4)))
print('Accuracy Score: ' + str(round(acc3, 4)))
print('')
print(cmtx3)
print('')
print(classification_report_imbalanced(y_test, y_pred3))

Cluster Centroid

Balanced Accuracy Score: 0.6546
Accuracy Score: 0.685

                  Predicted High Risk  Predicted Low Risk
Actual High Risk                   72                  29
Actual Low Risk                  6356               10748

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.01      0.71      0.63      0.02      0.67      0.45       101
   low_risk       1.00      0.63      0.71      0.77      0.67      0.44     17104

avg / total       0.99      0.63      0.71      0.77      0.67      0.44     17205



# Combination (Over and Under) Sampling

In this section, you will test a combination over- and under-sampling algorithm to determine if the algorithm results in the best performance compared to the other sampling algorithms above. You will resample the data using the SMOTEENN algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [85]:
# Resample the training data with SMOTEENN
# Warning: This is a large dataset, and this step may take some time to complete
from imblearn.combine import SMOTEENN
smote_enn = SMOTEENN(random_state=0)
X_resampled4, y_resampled4 = smote_enn.fit_resample(X, y)

In [92]:
# Train the Logistic Regression model using the resampled data
regressor.fit(X_resampled4, y_resampled4)

LogisticRegression(random_state=1)

In [93]:
# Calculated the balanced accuracy score
y_pred4 = regressor.predict(X_test)

bacc4 = (balanced_accuracy_score(y_test, y_pred4))
acc4 = accuracy_score(y_test,y_pred4)
print('Balanced Accuracy Score: ' + str(round(bacc4, 4)))
print('Accuracy Score: ' + str(round(acc4, 4)))

Balanced Accuracy Score: 0.6466
Accuracy Score: 0.5615


In [94]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred4)

array([[  74,   27],
       [7517, 9587]])

In [95]:
cmtx4 = pd.DataFrame(
    confusion_matrix(y_test, y_pred4, labels=['high_risk', 'low_risk']), 
    index=['Actual High Risk', 'Actual Low Risk'], 
    columns=['Predicted High Risk', 'Predicted Low Risk']
    )
print(cmtx4)

                  Predicted High Risk  Predicted Low Risk
Actual High Risk                   74                  27
Actual Low Risk                  7517                9587


In [96]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred4))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.01      0.73      0.56      0.02      0.64      0.42       101
   low_risk       1.00      0.56      0.73      0.72      0.64      0.40     17104

avg / total       0.99      0.56      0.73      0.71      0.64      0.40     17205



In [102]:
print('SMOTTEEN')
print('')
print('Balanced Accuracy Score: ' + str(round(bacc4, 4)))
print('Accuracy Score: ' + str(round(acc4, 4)))
print('')
print(cmtx4)
print('')
print(classification_report_imbalanced(y_test, y_pred4))

SMOTTEEN

Balanced Accuracy Score: 0.6466
Accuracy Score: 0.5615

                  Predicted High Risk  Predicted Low Risk
Actual High Risk                   74                  27
Actual Low Risk                  7517                9587

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.01      0.73      0.56      0.02      0.64      0.42       101
   low_risk       1.00      0.56      0.73      0.72      0.64      0.40     17104

avg / total       0.99      0.56      0.73      0.71      0.64      0.40     17205

